In [1]:
#!pip3 install .

In [2]:
import torch

In [3]:
from alpaca_lora_4bit.autograd_4bit import load_llama_model_4bit_low_ram, Autograd4bitQuantLinear, switch_backend_to

switch_backend_to("cuda")

Triton not found. Please run "pip install triton".

c:\Users\alex4321\anaconda3\envs\llama\lib\site-packages\tqdm-4.65.0-py3.10.egg\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Using CUDA implementation.


In [4]:
from alpaca_lora_4bit import autograd_4bit
from alpaca_lora_4bit import matmul_utils_4bit as mm4b

In [5]:
model, tokenizer = load_llama_model_4bit_low_ram("vicuna-7B-GPTQ-4bit-128g",
                                                 "vicuna-7B-GPTQ-4bit-128g/vicuna-7B-GPTQ-4bit-128g.safetensors",
                                                 groupsize=128)

Loading Model ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The safetensors archive passed at vicuna-7B-GPTQ-4bit-128g/vicuna-7B-GPTQ-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loaded the model in 4.50 seconds.


In [6]:
model.half()
for n, m in model.named_modules():
    if isinstance(m, Autograd4bitQuantLinear):
        if m.is_v1_model:
            m.zeros = m.zeros.half()
        m.scales = m.scales.half()
        m.bias = m.bias.half()
model.tie_weights()

In [7]:
from alpaca_lora_4bit.amp_wrapper import AMPWrapper
wrapper = AMPWrapper(model)
wrapper.apply_generate()

In [8]:
prompt = '''I think the meaning of life is'''
batch = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
batch = {k: v.cuda() for k, v in batch.items()}

In [9]:
with torch.no_grad():
     generated = model.generate(inputs=batch["input_ids"],
                                do_sample=True, use_cache=True,
                                repetition_penalty=1.1,
                                max_new_tokens=25,
                                temperature=1.1,
                                top_p=0.97,
                                top_k=40,
                                return_dict_in_generate=True,
                                output_attentions=False,
                                output_hidden_states=False,
                                output_scores=False)
tokenizer.decode(generated['sequences'][0])

'I think the meaning of life iswise\x97HARarkac\x9darsâE\x92laus_ "_ows _AMbrekereseller__\\chmarkuzz'

1. 13b - pt : 'I think the meaning of life is whatever each individual gives it. Votes: 0\nThere can be no friendship without confidence, and no confidence without integrity'
2. 7b - pt 'I think the meaning of life is\x92olateH centANTE susHI голов transferesac_AMHE_Arteller armsbreâows_ Modern'
3. 7b - safetensors 'I think the meaning of life iswise\x97HARarkac\x9darsâE\x92laus_ "_ows _AMbrekereseller__\\chmarkuzz'